In [111]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [74]:
import pandas as pd
import numpy as np
from sentence_transformers import InputExample, SentenceTransformer, models, losses, evaluation, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
from scipy import spatial
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from data_processing import get_data

In [75]:
x_train, x_test1, x_test2, x_test3, x_test4, x_test5, y_train, y_test1, y_test2, y_test3, y_test4, y_test5 = get_data()
def convert_list_data_into_InputExample(x, y):
    examples = []
    for s, label in zip(x, y):
        question1, question2 = s
        examples.append(InputExample(texts=[question1, question2], label=float(label)))
    return examples

train = convert_list_data_into_InputExample(x_train, y_train)
test1 = convert_list_data_into_InputExample(x_test1, y_test1)
test2 = convert_list_data_into_InputExample(x_test2, y_test2)
test3 = convert_list_data_into_InputExample(x_test3, y_test3)
test4 = convert_list_data_into_InputExample(x_test4, y_test4)
test5 = convert_list_data_into_InputExample(x_test5, y_test5)

In [76]:
vectorizer = CountVectorizer()
sample = np.array(x_test2)
sample = sample.tolist()

In [77]:
def f(number):
    return float(number)

In [78]:
def count_vector_similarity(sample):
    n = len(sample)
    simi = []
    vector_list1=[]
    vector_list2=[]
    words_list=[]
    for i in range(n):
        fit = vectorizer.fit_transform(sample[i]).todense()
        vector1 = fit[0].tolist()[0]
        vector2 = fit[1].tolist()[0]
        float_numbers_iterator1 = map(f, vector1)
        float_numbers_iterator2 = map(f, vector2)
        vector1 = list(float_numbers_iterator1)
        vector2 = list(float_numbers_iterator2)
        vector_list1.append(vector1)
        vector_list2.append(vector2)
        words = vectorizer.get_feature_names_out()
        words_list.append(words)
        similarity = util.cos_sim(vector1, vector2)
        simi.append(similarity)
    return simi, vector_list1, vector_list2, words_list

In [79]:
def f1_score(simi, y):
    TP = 0
    FP = 0
    FN = 0
    prediction_label = [0]*len(simi)
    for i in range(len(simi)):
        if simi[i] > 0.5:
            prediction_label[i]=1
            if y[i]=='1':
                TP+=1
            elif y[i]=='0':
                FP+=1
        else:
            if y[i]=='1':
                FN+=1
    R=TP/(TP+FN)
    P=TP/(TP+FP)
    f1_score1=2*P*R/(P+R)
    return f1_score1

In [80]:
simi_list, vector1, vector2, word_list = count_vector_similarity(sample)
f = f1_score(simi_list, y_test2)
print(f)

0.6083810941570525
